In [1]:
import pandas as pd
import os
import glob

In [2]:
all_files = glob.glob(os.path.join("data/raw/for_training/" , "*.csv"))

In [3]:
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [4]:
frame

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0
...,...,...,...,...,...,...,...,...,...,...,...
117522,2020-09,YISHUN,5 ROOM,716,YISHUN ST 71,07 TO 09,131.0,Improved,1987,66 years 03 months,440000.0
117523,2020-09,YISHUN,5 ROOM,760,YISHUN ST 72,07 TO 09,122.0,Improved,1987,65 years 06 months,458000.0
117524,2020-09,YISHUN,5 ROOM,835,YISHUN ST 81,04 TO 06,122.0,Improved,1987,66 years 04 months,490000.0
117525,2020-09,YISHUN,EXECUTIVE,791,YISHUN AVE 2,04 TO 06,146.0,Maisonette,1987,66 years 03 months,558000.0


In [8]:
sorted(frame["storey_range"].unique())

['01 TO 03',
 '04 TO 06',
 '07 TO 09',
 '10 TO 12',
 '13 TO 15',
 '16 TO 18',
 '19 TO 21',
 '22 TO 24',
 '25 TO 27',
 '28 TO 30',
 '31 TO 33',
 '34 TO 36',
 '37 TO 39',
 '40 TO 42',
 '43 TO 45',
 '46 TO 48',
 '49 TO 51']

In [9]:
sorted(frame["storey_range"].unique()).astype('category').cat.codes

AttributeError: 'list' object has no attribute 'astype'

In [16]:
pd.to_datetime(frame["month"])

0        2015-01-01
1        2015-01-01
2        2015-01-01
3        2015-01-01
4        2015-01-01
            ...    
117522   2020-09-01
117523   2020-09-01
117524   2020-09-01
117525   2020-09-01
117526   2020-09-01
Name: month, Length: 117527, dtype: datetime64[ns]

In [3]:
cpi = pd.read_csv("data/for_data_cleaning/cpi.csv")

In [5]:
cpi.head()

,month,cpi
0,1961 Jan,26.553
1,1961 Feb,26.669
2,1961 Mar,26.553
3,1961 Apr,26.650
4,1961 May,26.728


In [6]:
cpi["month"] = pd.to_datetime(cpi["month"])
cpi

,month,cpi
0,1961-01-01,26.553
1,1961-02-01,26.669
2,1961-03-01,26.553
3,1961-04-01,26.650
4,1961-05-01,26.728
...,...,...
712,2020-05-01,100.498
713,2020-06-01,100.487
714,2020-07-01,98.266
715,2020-08-01,99.849


In [24]:
import json
import requests

def find_postal(add):
    '''With the block number and street name, get the full address of the hdb flat,
    including the postal code, geogaphical coordinates (lat/long)'''
    
    # Do not need to change the URL
    url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+ add        
    
    # Retrieve information from website
    response = requests.get(url)
    try:
        data = json.loads(response.text) 
    except ValueError:
        print('JSONDecodeError')
        pass

    result = data["results"][0]
    latitude = float(result["LATITUDE"])
    longitude = float(result["LONGITUDE"])

    return latitude, longitude
    
    # if len(data["results"]) != 0:
        
    #     temp_df = pd.DataFrame([data["results"][0]])
    #     # The "add" is the address that was used to search in the website
    #     temp_df["address"] = add
    
    #     # Create the file with the first row that is read in 
    #     if index == 0:
    #         file = temp_df
    #     else:
    #         file = file.append(temp_df)
                
    # return file[["address","LATITUDE","LONGITUDE"]]


In [25]:
add = "174 ANG MO KIO AVE 4"
longitude, latitude = find_postal(add)



In [26]:
longitude

1.37509746867904

In [27]:
latitude

103.83761896123

In [34]:
test_frame = frame.head()
test_frame

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0


In [38]:
test_frame["coordinates"] = test_frame.apply(lambda x: find_postal(x["block"] + " " + x["street_name"]), axis = 1)
test_frame

C:\Users\angus\AppData\Local\Temp\ipykernel_30716\2948387337.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_frame["coordinates"] = test_frame.apply(lambda x: find_postal(x["block"] + " " + x["street_name"]), axis = 1)


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,coordinates
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,"(1.37509746867904, 103.83761896123)"
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0,"(1.37392238703482, 103.855621370524)"
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0,"(1.37354853919927, 103.838176471398)"
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0,"(1.36776095130953, 103.855357145908)"
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0,"(1.3716257020332, 103.857736107527)"


In [39]:
test_frame["longitude"] = test_frame["coordinates"].apply(lambda x: x[0])
test_frame["latitude"] = test_frame["coordinates"].apply(lambda x: x[1])
test_frame

C:\Users\angus\AppData\Local\Temp\ipykernel_30716\3236176125.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_frame["longitude"] = test_frame["coordinates"].apply(lambda x: x[0])
C:\Users\angus\AppData\Local\Temp\ipykernel_30716\3236176125.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_frame["latitude"] = test_frame["coordinates"].apply(lambda x: x[1])


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,coordinates,longitude,latitude
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,"(1.37509746867904, 103.83761896123)",1.375097,103.837619
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0,"(1.37392238703482, 103.855621370524)",1.373922,103.855621
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0,"(1.37354853919927, 103.838176471398)",1.373549,103.838176
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0,"(1.36776095130953, 103.855357145908)",1.367761,103.855357
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0,"(1.3716257020332, 103.857736107527)",1.371626,103.857736


In [41]:
coordinates = test_frame.apply(lambda x: find_postal(x["block"] + " " + x["street_name"]), axis = 1)

,0
0,"(1.37509746867904, 103.83761896123)"
1,"(1.37392238703482, 103.855621370524)"
2,"(1.37354853919927, 103.838176471398)"
3,"(1.36776095130953, 103.855357145908)"
4,"(1.3716257020332, 103.857736107527)"


In [46]:
pd.DataFrame(coordinates.tolist(), columns = ["column_1", "column_2"])

,column_1,column_2
0,1.375097,103.837619
1,1.373922,103.855621
2,1.373549,103.838176
3,1.367761,103.855357
4,1.371626,103.857736


In [11]:
mrt = pd.read_csv("data/for_feature_engineering/mrt_stations/mrt_station_coordinates_w_period.csv", index_col=None, header=0)
mrt

,Name,LATITUDE,LONGITUDE,Opening year,Opening month
0,SERANGOON MRT STATION,1.350634,103.872772,2003,6
1,TAMPINES WEST MRT STATION,1.345515,103.938437,2017,10
2,BOON KENG MRT STATION,1.319940,103.862053,2003,6
3,MAYFLOWER MRT STATION,1.371463,103.836568,2021,8
4,TAN KAH KEE MRT STATION,1.326392,103.806548,2015,12
...,...,...,...,...,...
122,GUL CIRCLE MRT STATION,1.319471,103.660530,2017,6
123,CHINATOWN MRT STATION,1.284224,103.845144,2003,6
124,NEWTON MRT STATION,1.312319,103.837985,1987,12
125,NOVENA MRT STATION,1.320441,103.843826,1987,12


In [5]:
mrt.dtypes

Name              object
LATITUDE         float64
LONGITUDE        float64
Opening year       int64
Opening month      int64
dtype: object

In [12]:
mrt = mrt.rename(columns={"Opening year": "YEAR", "Opening month": "MONTH"})

In [9]:
mrt[['YEAR', 'MONTH']].assign(DAY=1)

,YEAR,MONTH,DAY
0,2003,6,1
1,2017,10,1
2,2003,6,1
3,2021,8,1
4,2015,12,1
...,...,...,...
122,2017,6,1
123,2003,6,1
124,1987,12,1
125,1987,12,1


In [14]:
pd.to_datetime(mrt[['YEAR', 'MONTH']].assign(DAY=1))

0     2003-06-01
1     2017-10-01
2     2003-06-01
3     2021-08-01
4     2015-12-01
         ...    
122   2017-06-01
123   2003-06-01
124   1987-12-01
125   1987-12-01
126   2021-08-01
Length: 127, dtype: datetime64[ns]

In [10]:
regions={"North":["WOODLANDS","SEMBAWANG","YISHUN","BUKIT BATOK"],
                "Northeast":["HOUGANG","SENGKANG","PUNGGOL","ANG MO KIO","SERANGOON"],
                "Central":["BUKIT MERAH","CENTRAL AREA","BISHAN","GEYLANG","KALLANG/WHAMPOA","QUEENSTOWN","TOA PAYOH","MARINE PARADE"],
                "West":["JURONG WEST","JURONG EAST","CLEMENTI","BUKIT PANJANG","BUKIT TIMAH","CHOA CHU KANG","LIM CHU KANG"],
                "East":["PASIR RIS","TAMPINES","BEDOK"]}

region_feature = 'region'
town_feature  = 'town'


ValueError: too many values to unpack (expected 2)

In [13]:
{town: region for region, towns in regions.items() for town in towns}

{'WOODLANDS': 'North',
 'SEMBAWANG': 'North',
 'YISHUN': 'North',
 'BUKIT BATOK': 'North',
 'HOUGANG': 'Northeast',
 'SENGKANG': 'Northeast',
 'PUNGGOL': 'Northeast',
 'ANG MO KIO': 'Northeast',
 'SERANGOON': 'Northeast',
 'BUKIT MERAH': 'Central',
 'CENTRAL AREA': 'Central',
 'BISHAN': 'Central',
 'GEYLANG': 'Central',
 'KALLANG/WHAMPOA': 'Central',
 'QUEENSTOWN': 'Central',
 'TOA PAYOH': 'Central',
 'MARINE PARADE': 'Central',
 'JURONG WEST': 'West',
 'JURONG EAST': 'West',
 'CLEMENTI': 'West',
 'BUKIT PANJANG': 'West',
 'BUKIT TIMAH': 'West',
 'CHOA CHU KANG': 'West',
 'LIM CHU KANG': 'West',
 'PASIR RIS': 'East',
 'TAMPINES': 'East',
 'BEDOK': 'East'}

In [36]:
import sqlalchemy
import os
os.environ['POSTGRES_USER'] = "postgres"
os.environ['POSTGRES_PWD'] = "postgres"
os.environ['POSTGRES_HOST'] = "127.0.0.1"
os.environ['POSTGRES_PORT'] = "5433"
os.environ['POSTGRES_DB'] = "hdb_project"


url_object = sqlalchemy.engine.URL.create(
        "postgresql+psycopg2",
        username=os.getenv("POSTGRES_USER"),
        password=os.getenv("POSTGRES_PWD"),
        host=os.getenv("POSTGRES_HOST"),
        port=os.getenv("POSTGRES_PORT"),
        database=os.getenv("POSTGRES_DB"),
    )


In [37]:
engine = sqlalchemy.create_engine(url_object, pool_size=5, pool_recycle=3600)


In [38]:
engine.begin()

In [92]:
import pandas as pd
query = "SELECT * FROM hdb_training_features"
df = pd.read_sql(query, engine)

In [94]:
df_obj = df.select_dtypes("object")
df[df_obj.columns] = df_obj.astype(str).apply(lambda x: x.str.rstrip())

Index(['town', 'flat_type', 'block', 'street_name', 'storey_range',
       'flat_model', 'remaining_lease', 'region', 'year_month',
       'date_context'],
      dtype='object')

In [97]:
df[df_obj.columns] = df_obj.astype(str).apply(lambda x: x.str.rstrip())

In [33]:
df = df[df['id']==181]

In [34]:
df_dict = df.iloc[0].to_dict()

In [99]:
derived_input_data = df.iloc[0].to_dict()

In [100]:
derived_input_data

{'id': 181,
 'month': 1,
 'town': 'BEDOK',
 'flat_type': '3 ROOM',
 'block': '548',
 'street_name': 'BEDOK NTH AVE 1',
 'storey_range': '04 TO 06',
 'floor_area_sqm': 68.0,
 'flat_model': 'NEW GENERATION',
 'lease_age': 35,
 'latitude': 1.33131021834565,
 'longitude': 103.92685391703,
 'no_of_malls_within_2_km': 3,
 'distance_to_nearest_malls': 0.6478704253059351,
 'no_of_schools_within_2_km': 16,
 'distance_to_nearest_schools': 0.5654799761533605,
 'no_of_parks_within_2_km': 2,
 'distance_to_nearest_parks': 0.6066781831238272,
 'no_of_MRT_stations_within_2_km': 2,
 'distance_to_nearest_MRT_stations': 1.9196800461638075,
 'lease_commence_date': 1980,
 'remaining_lease': '64',
 'resale_price': 276926.6501701759,
 'cpi': 111.943,
 'region': 'East',
 'year_month': '2015-01-01',
 'year': 2015,
 'date_context': '2023-05-13'}

In [102]:
import json
json.dumps(derived_input_data, default=str)

'{"id": 181, "month": 1, "town": "BEDOK", "flat_type": "3 ROOM", "block": "548", "street_name": "BEDOK NTH AVE 1", "storey_range": "04 TO 06", "floor_area_sqm": 68.0, "flat_model": "NEW GENERATION", "lease_age": 35, "latitude": 1.33131021834565, "longitude": 103.92685391703, "no_of_malls_within_2_km": 3, "distance_to_nearest_malls": 0.6478704253059351, "no_of_schools_within_2_km": 16, "distance_to_nearest_schools": 0.5654799761533605, "no_of_parks_within_2_km": 2, "distance_to_nearest_parks": 0.6066781831238272, "no_of_MRT_stations_within_2_km": 2, "distance_to_nearest_MRT_stations": 1.9196800461638075, "lease_commence_date": 1980, "remaining_lease": "64", "resale_price": 276926.6501701759, "cpi": 111.943, "region": "East", "year_month": "2015-01-01", "year": 2015, "date_context": "2023-05-13"}'

In [16]:
ordinal = df[["storey_range","region"]]
ordinal

,storey_range,region
0,04 TO 06 ...,East ...
1,04 TO 06 ...,East ...
2,01 TO 03 ...,East ...
3,04 TO 06 ...,East ...
4,04 TO 06 ...,East ...
...,...,...
115841,07 TO 09 ...,East ...
115842,01 TO 03 ...,East ...
115843,01 TO 03 ...,East ...
115844,01 TO 03 ...,East ...


In [22]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
pd.DataFrame(encoder.fit_transform(ordinal), columns=encoder.get_feature_names_out(ordinal.columns), index=ordinal.index,)


,storey_range,region
0,1.0,1.0
1,1.0,1.0
2,0.0,1.0
3,1.0,1.0
4,1.0,1.0
...,...,...
115841,2.0,1.0
115842,0.0,1.0
115843,0.0,1.0
115844,0.0,1.0


In [15]:
ordinal.drop_duplicates().sort_values(by="storey_range")

,storey_range,encoded
2,01 TO 03 ...,0.0
0,04 TO 06 ...,1.0
9,07 TO 09 ...,2.0
5,10 TO 12 ...,3.0
24,13 TO 15 ...,4.0
21,16 TO 18 ...,5.0
42,19 TO 21 ...,6.0
81,22 TO 24 ...,7.0
187,25 TO 27 ...,8.0
194,28 TO 30 ...,9.0


In [21]:
df.select_dtypes(include=['object']).columns.tolist()

['town',
 'flat_type',
 'block',
 'street_name',
 'storey_range',
 'flat_model',
 'remaining_lease',
 'region',
 'year_month',
 'date_context']

In [3]:
from sklearn import metrics
print(metrics.get_scorer_names())

['accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'balanced_accuracy', 'completeness_score', 'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 'homogeneity_score', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted', 'matthews_corrcoef', 'max_error', 'mutual_info_score', 'neg_brier_score', 'neg_log_loss', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_gamma_deviance', 'neg_mean_poisson_deviance', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'neg_negative_likelihood_ratio', 'neg_root_mean_squared_error', 'normalized_mutual_info_score', 'positive_likelihood_ratio', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'rand_score', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc', 'roc_auc_ovo', 'roc_auc_ovo_weight

In [14]:
import mlflow
import os
import joblib


os.environ['MLFLOW_TRACKING_URI'] = "http://127.0.0.1:5005"
os.environ['MODEL_URI'] = "003bd90b6c0145f380cc7ee7701d7624"
os.environ['RUN_ID'] = "194177493496013876"



In [15]:


# Load model
mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])
model_uri = os.getenv("MODEL_URI")
run_id =  os.getenv("RUN_ID")
artifact_uri = f'mlflow-artifacts:/{run_id}/{model_uri}/artifacts/model'
artifact_uri

'mlflow-artifacts:/194177493496013876/003bd90b6c0145f380cc7ee7701d7624/artifacts/model'

In [16]:
try:
    mlflow.artifacts.download_artifacts(
        artifact_uri=artifact_uri, dst_path="../models"
    )
except Exception as mlflow_error:
    raise mlflow_error


In [21]:
import json
features = json.load(open('./models/artifacts/features.json'))


In [11]:
features

{'features': ['month',
  'year',
  'floor_area_sqm',
  'lease_age',
  'no_of_malls_within_2_km',
  'distance_to_nearest_malls',
  'no_of_schools_within_2_km',
  'distance_to_nearest_schools',
  'no_of_parks_within_2_km',
  'distance_to_nearest_parks',
  'no_of_MRT_stations_within_2_km',
  'distance_to_nearest_MRT_stations',
  'storey_range',
  'flat_type_3 ROOM',
  'flat_type_4 ROOM',
  'flat_type_5 ROOM',
  'flat_type_EXECUTIVE',
  'flat_model_APARTMENT',
  'flat_model_MAISONETTE',
  'flat_model_MODEL A',
  'flat_model_NEW GENERATION',
  'flat_model_SPECIAL',
  'flat_model_STANDARD',
  'region_Central',
  'region_East',
  'region_North',
  'region_Northeast',
  'region_West']}

In [22]:
import hdb_resale_estimator

builder = joblib.load(model_path)

0it [00:00, ?it/s]
c:\Users\angus\Anaconda3\envs\hdb-env\lib\site-packages\shap\utils\_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
c:\Users\angus\Anaconda3\envs\hdb-env\lib\site-packages\shap\utils\_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_mi

In [1]:
col = "nearest_MRT_stations_coordinates"
print(col.replace("nearest_","").replace("_coordinates",""))

MRT_stations


In [2]:
tuple([1,3])

(1, 3)

In [3]:
date = "2020-09"

In [7]:
from datetime import datetime
datetime.strptime(date, '%Y-%m').date().year

2020

In [ ]:
year = 1994

In [ ]:
from datetime import datetime
datetime.strptime(date, '%Y-%m').date()

In [12]:
import requests
import json
def find_coordinates(add: str) -> tuple:
    """With the block number and street name, get the full address of the hdb flat,
    including the postal code, geogaphical coordinates (lat/long)

    Args:
        add (str): block number and street name

    Returns:
        tuple: latitude and longitude coordinates
    """
    # Do not need to change the URL
    url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+ add        
    
    # Retrieve information from website
    response = requests.get(url)
    try:
        data = json.loads(response.text) 
    except ValueError:
        print('JSONDecodeError')
        pass

    if len(data["results"]) != 0:
        result = data["results"][0]
        latitude, longitude = float(result["LATITUDE"]), float(result["LONGITUDE"])

    else:
        latitude, longitude = float("inf"), float("inf")

    return latitude, longitude

In [16]:
add = "hfdbawdhgbdlk"

In [18]:
find_coordinates(add)

(inf, inf)

In [1]:
from fastapi import FastAPI
import logging
import mlflow
import os
import joblib
import pandas as pd
import glob
import yaml

import hdb_resale_estimator as hdb_est

os.environ['MLFLOW_TRACKING_URI'] = "http://127.0.0.1:5005"
os.environ['MODEL_URI'] = "d91f68fb4c6e41c3be75093d4a40a934"
os.environ['RUN_ID'] = "194177493496013876"

# Load model
mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])
model_uri = os.getenv("MODEL_URI")
run_id =  os.getenv("RUN_ID")
artifact_uri = f'mlflow-artifacts:/{run_id}/{model_uri}/artifacts/model'
try:
    mlflow.artifacts.download_artifacts(
        artifact_uri=artifact_uri, dst_path="models"
    )
except Exception as mlflow_error:
    raise mlflow_error


model_path = glob.glob("../models/model/*.joblib")[0]
builder = joblib.load(model_path)


0it [00:00, ?it/s]
c:\Users\angus\Anaconda3\envs\hdb-env\lib\site-packages\shap\utils\_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
c:\Users\angus\Anaconda3\envs\hdb-env\lib\site-packages\shap\utils\_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_mi

In [2]:
builder.objects

{'features': ['month',
  'year',
  'flat_type',
  'storey_range',
  'floor_area_sqm',
  'flat_model',
  'lease_age',
  'no_of_malls_within_2_km',
  'distance_to_nearest_malls',
  'no_of_schools_within_2_km',
  'distance_to_nearest_schools',
  'no_of_parks_within_2_km',
  'distance_to_nearest_parks',
  'no_of_MRT_stations_within_2_km',
  'distance_to_nearest_MRT_stations',
  'region'],
 'ordinal_encoder': {'columns': Index(['storey_range'], dtype='object'),
  'encoder': OrdinalEncoder()},
 'one_hot_encoder': {'columns': ['flat_type', 'flat_model', 'region'],
  'encoder': OneHotEncoder()},
 'explainer': <shap.explainers._permutation.Permutation at 0x1b84b5f3b20>}

In [3]:
PRED_MODEL_EXPLAINER = builder.objects["explainer"]

In [10]:
input_data = {'id': 1,
                'month': '2015-01',
                'town': 'ANG MO KIO',
                'flat_type': '3 ROOM',
                'block': '405',
                'street_name': 'ANG MO KIO AVE 10',
                'storey_range': '01 TO 03',
                'floor_area_sqm': 186.0,
                'flat_model': '2-room', 
                'lease_commence_date': 1994,
                'remaining_lease': ''}

In [8]:
import yaml

with open('../conf/data_prep.yaml', 'r') as file:
    config = yaml.safe_load(file)



In [11]:

input_data = pd.DataFrame([input_data])
data_cleaner = hdb_est.data_prep.data_cleaning.DataCleaner(raw_hdb_data=input_data,
                                                            params=config["data_prep"],
                                                            inference_mode=True)
clean_input_data = data_cleaner.clean_data()

feature_engineer = hdb_est.data_prep.feature_engineering.FeatureEngineer(
    params=config["data_prep"], inference_mode=True
)

derived_input_data = feature_engineer.engineer_features(
    hdb_data=clean_input_data)

derived_input_data

100%|██████████| 1/1 [00:00<00:00,  5.64it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'data/for_feature_engineering/malls/mall_coordinates.csv'

In [ ]:
shap_values = explainer(train_X)